# Updating facts (on the fly)

This notebook is going to show the same process than the `updating_documents` notebook, but with facts instead. Nothing fancy, just to show you how it works.

In [1]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Action(
    id = "search_facts",
    purpose = "Search for relevant facts to answer the Objective's question",
    tool = "FactSearch",
    prompt = "Please use the Objective's question to infer the search query",
))

main.add(gp.Decision(
    id = "check_context",
    purpose = "Check if the answer to the Objective's question is in your Context",
    question = "Is the answer to the Objective's question in your context?"
))

main.add(gp.Action(
    id = "answer_context_based",
    purpose = "Answer to the Objective's question based on your Context",
    tool = "Speak",
    prompt = "Answer to the Objective's question, if there is relevant information in your Context, please use it",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer to the Objective's question",
    tool = "Speak",
    prompt = "Answer to the Objective's question, don't say it is based on your search just answer",
))

main.add(gp.Action(
    id = "save_answer",
    purpose = "Save the answer to the Objective Question into memory",
    tool = "AddFact",
    prompt = "Use the answer in your context to infer the document to save, never explain what you are doing",
))

main.connect("start", "search_facts")
main.connect("search_facts", "check_context")
main.connect("check_context", "answer_context_based", label="Yes")
main.connect("check_context", "answer", label="No")
main.connect("answer", "save_answer")
main.connect("answer_context_based", "end")
main.connect("save_answer", "end")

main.build()

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# We add the programs into memory
from hybridagi.memory.integration.local import LocalProgramMemory, LocalFactMemory

program_memory = LocalProgramMemory(index_name="update_facts")

program_memory.update(main)

# Then we instanciate the document memory for later use

fact_memory = LocalFactMemory(index_name="update_facts")

In [3]:
# Now we setup the agent and tools
import dspy
from hybridagi.core.datatypes import AgentState
from hybridagi.core.pipeline import Pipeline
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.splitters import DocumentSentenceSplitter
from hybridagi.modules.embedders import FactEmbedder
from hybridagi.modules.extractors import LLMFactExtractor
from hybridagi.modules.deduplicators import EntityDeduplicator
from hybridagi.modules.retrievers.integration.local import FAISSFactRetriever 
from hybridagi.modules.agents.tools import (
    SpeakTool,
    FactSearchTool,
    AddFactTool,
)

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384,
)

pipeline = Pipeline()

# The facts processing pipeline, takes an input document and convert it to facts

pipeline.add("chunk_docs", DocumentSentenceSplitter(
    method = "word",
    chunk_size = 100,
    chunk_overlap = 0,
    separator = " ",
))
pipeline.add("extract_facts", LLMFactExtractor())
pipeline.add("deduplicate_entities", EntityDeduplicator(method="exact"))
pipeline.add("embed_facts", FactEmbedder(embeddings=embeddings))

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    FactSearchTool(
        retriever = FAISSFactRetriever(
            fact_memory = fact_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 1.0,
            k = 5,
            reranker = None,
        ),
    ),
    AddFactTool(
        fact_memory = fact_memory,
        pipeline = pipeline, # Here we bind the document processing pipeline to our tool
    ),
]

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

In [4]:
from hybridagi.core.datatypes import Query

# Now we can test our system
result = agent(Query(query="What the definition of a neuro symbolic AI"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: What the definition of a neuro symbolic AI
--- Step 1 ---
Action Purpose: Search for relevant facts to answer the Objective's question
Action: {
  "query": "definition of neuro symbolic AI",
  "facts": []
}
--- Step 2 ---
Decision Purpose: Check if the answer to the Objective's question is in your Context
Decision Question: Is the answer to the Objective's question in your context?
Decision: NO
--- Step 3 ---
Action Purpose: Answer to the Objective's question
Action: {
  "message": "A neuro-symbolic AI is a type of artificial intelligence that combines symbolic AI and neural networks. It aims to bridge the gap between the two approaches by allowing neural networks to learn complex representations of the world while also being able to reason about those representations using symbolic logic. This integration allows the AI to learn from data, as well as make decisions based on explicit rules and reasoning."
}


100%|██████████| 5/5 [00:00<00:00, 19.73it/s]

--- Step 4 ---
Action Purpose: Save the answer to the Objective Question into memory
Action: {
  "facts": [
    {
      "fact": "(:AIType {name:\"Neuro-symbolic\"})-[:IS]->(:ArtificialIntelligence {name:\"Artificial Intelligence\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:COMPOSES]->(:NeuralNetworks {name:\"Neural Networks\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:COMPOSES]->(:SymbolicLogic {name:\"Symbolic Logic\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:ALLOWS]->(:LearnData {name:\"Learn from Data\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:ENABLES]->(:ReasonExplicitRules {name:\"Make decisions based on explicit rules and reasoning\"})"
    }
  ]
}
--- Step 5 ---
End Program: main
A neuro-symbolic AI is a type of artificial intelligence that combines symbolic AI and neural networks. It aims to bridge the gap between the two app

In [5]:
from hybridagi.core.datatypes import Query

# Now we can test our system
result = agent(Query(query="What the definition of a neuro symbolic AI"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: What the definition of a neuro symbolic AI
--- Step 1 ---
Action Purpose: Search for relevant facts to answer the Objective's question
Action: {
  "query": "definition of neuro symbolic AI",
  "facts": [
    {
      "fact": "(:AIType {name:\"Neuro-symbolic\"})-[:IS]->(:ArtificialIntelligence {name:\"Artificial Intelligence\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:COMPOSES]->(:SymbolicLogic {name:\"Symbolic Logic\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:COMPOSES]->(:NeuralNetworks {name:\"Neural Networks\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:ENABLES]->(:ReasonExplicitRules {name:\"Make decisions based on explicit rules and reasoning\"})"
    },
    {
      "fact": "(:NeuroSymbolicAI {name:\"Neuro-symbolic AI\"})-[:ALLOWS]->(:LearnData {name:\"Learn from Data\"})"
    }
  ]
}
--- Step 2 ---
Decision Purpose: Ch